In [11]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from time import time
import sklearn
from sklearn.covariance import LedoitWolf, OAS, MinCovDet
from scipy.stats import wasserstein_distance
import math
import transformers
import textattack
from textattack.datasets import HuggingFaceDataset
from textattack.goal_functions import UntargetedClassification
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack import Attack
from textattack.transformations import WordSwap
from datasets import load_dataset

In [12]:
def get_train_data(name="ag_news"):
    dataset = load_dataset(name).shuffle(seed=0)
    num_labels = dataset["train"].features["label"].num_classes

In [5]:
dataset = load_dataset("ag_news")
num_labels = 4
# dataset.shuffle(seed=0)

Using custom data configuration default


Dataset ag_news downloaded and prepared to /home/onyxia/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

# Etapes du Projet:

A. RECUPERER LES DONNEES
1. Récupérer les données:\
    a. Récupérer les csv -> Fait\
    b. On n'a que des attaques/données qui ont été attaquées, récupérer d'autres données random ou se              contenter de ça?
2. Récupérer BERT -> à Faire

B. Preprocessing
1. Data cleaning (drop duplicates, supprimer les attaques qui sont les mêmes que le text de base, etc...à
2. Tokenizer
3. Réduire la dimension

C. COMPARER LES MODELES
1. Aller à l'intérieur de BERT
2. Coder les différents scores (Manhalobis, max(softmax), KL(softmax,uniform), éventuellement d'autres options qu'on veut test)
3. Calculer les seuils (par exemple set False Positive Rate à 10%, c'est ce qu'ils font pour Manhalobis)
4. Montrer les résultats (ROC, TPR, TNR, Confusion Matrix, etc...)


Description étape par étape du code Mahalanobis:\
STRUCTURE FINALE : (structure du main)\
1.	Prérequis (déjà faits)\
a.	Charge le modèle\
b.	Tokenize\
2.	Premier load :\
a.	Récupère data[‘train’]\
b.	En récupère 90% (10% de valset non utilisé)\
c.	Récupère les train_features dessus (ie dataset où data[label 1] = outputs(layer) qui ont mené au label 1)\
d.	On preprocess\
e.	On récupère mean,cov \
3.	Deuxième load :\
a.	On récupère un testset qui contient des phrases et attaque ou pas attaque\
b.	On récupère test_features (et préprocess) (qui contient les layers, à partir de testset[‘text’]\ 
c.	On calcul leur distance de mahalan (pour chaque estimateur) avec la mean,cov du train\
d.	On calcul la confidence (découle de mahalan, MLE)\
e.	On calcul le seuil de conf % fpr sur le TEST SET (fonction sklearn roc_curve, fonction un peu chelou à capter)\
f.	On affiche les autres métriques

#### Global Variables:
1. 
batch_size

## The idea:
predictor : $\mathbb{1}_{attack} = \mathbb{1}_{dist>threshold}$\
with $threshold$ such that $P($attack is predicted$|$not attack$)=0.1$\
$\begin{itemize}
\item Compute manhalobis distance -> we need x, mu, and sigma\
to get sigma we estimate cov matrix on train set -> we need output.hs
\item Compute threshold
$

In [ ]:
df_bae.head()

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,Fears for T [[N]] pension after talks Unions r...,Fears for T [[pl]] pension after talks Unions ...,0.000575,0.613322,2.0,3.0,2.0,84.0,Successful
1,The Race is [[On]]: [[Second]] Private Team Se...,The Race is [[over]]: [[private]] Private Team...,0.000504,0.000980,3.0,3.0,3.0,178.0,Failed
2,Ky. Company Wins Grant to Study Peptides (AP) ...,Ky. Company Wins Grant to Study Peptides (AP) ...,0.000354,0.000449,3.0,3.0,3.0,148.0,Failed
3,Prediction Unit Helps [[Forecast]] Wildfires (...,Prediction Unit Helps [[forecast]] Wildfires (...,0.000628,0.017141,3.0,3.0,3.0,139.0,Failed
4,Calif. Aims to Limit Farm-Related Smog (AP) [[...,Calif. Aims to Limit Farm-Related Smog (AP) [[...,0.000951,0.162635,3.0,3.0,3.0,52.0,Failed


## A. Récupération des données et du modèle

### A.1 Récupération des données

In [14]:
df_bae = pd.read_csv("bae.csv")
df_pwws = pd.read_csv("pwws.csv")
df_textfooler = pd.read_csv("textfooler.csv")
df_tf = pd.read_csv("tf-adj.csv")

In [15]:
df_bae.result_type.value_counts(1)
# Some attacks are skipped for unknown reasons as of now, \
# in those cases there are no attacks and the perturbed text is the original text

Failed        0.815395
Successful    0.136053
Skipped       0.048553
Name: result_type, dtype: float64

### A.2 Récupération du modèle

In [16]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-ag-news"
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "textattack/bert-base-uncased-ag-news"
)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
goal_function = UntargetedClassification(model_wrapper)
dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/home/onyxia/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


### Accessing BERT

Guide du github pour pénétrer BERT:\
Méthodes de réduction de dimension dans utils.miscelaneous\
pénétration dans utils.detection

layer = -1\
batch_size = 256

class model:
    def __init__(self,data,):
        self.data = data
        

In [144]:
def run_BERT(series, batch_size=10, output_hs=False, output_attentions=False):
    # We choose to run by batches to avoid kernel death
    tokens = tokenizer(
        series.tolist(),
        max_length=256,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        add_special_tokens=True,
        return_attention_mask=True,
    )
    num_batches = (len(series) - 1) // batch_size + 1
    Outputs = []
    for batch_idx in range(num_batches):  # Next line can probably be coded way cleaner
        window = [batch_idx * batch_size, (batch_idx + 1) * batch_size]
        Outputs.append(
            model(
                input_ids=tokens["input_ids"][window[0] : window[1]],
                attention_mask=tokens["attention_mask"][window[0] : window[1]],
                token_type_ids=tokens["token_type_ids"][window[0] : window[1]],
                output_hidden_states=output_hs,
                output_attentions=output_attentions,
            )
        )
    return Outputs
    # Outputs = torch.cat(outputs)
    # return Outputs

In [10]:
outputs = run_BERT(series[:11], output_hs=True, output_attentions=True)

NameError: name 'run_BERT' is not defined

In [146]:
# Idée : à terme tout mettre dans une classe, mettre des if last_layer : last_layer.append, etc...
# Et définir au début: if method = Manhalobis, set last_layer = True, etc...
def get_features(Outputs):
    # We wish to isolate a dataset containing in X the Manhalobis distance, and in Y the prediction
    last_layer = []
    pred = []
    softmax = []
    num_batches = len(Outputs)
    for batch_idx in range(num_batches):
        last_layer.append(Outputs[batch_idx].hidden_states[-1][:, 0, :])
        pred.append(Outputs[batch_idx].logits.argmax(1))
        softmax.append(torch.softmax(Outputs[batch_idx].logits, dim=1))
    last_layer = torch.cat(last_layer)
    pred = torch.cat(pred)
    softmax = torch.cat(softmax)
    return last_layer, pred, softmax

In [214]:
last_layer, pred, softmax = get_features(outputs)

In [513]:
def get_cov_estimator(x, name):
    if name == "LW":
        print(LedoitWolf().fit(x).shrinkage_)
        print(LedoitWolf().fit(x).mahalanobis(y.numpy().reshape(1, -1)))
        return LedoitWolf().fit(x).covariance_
    if name == "empirical":
        return np.cov(x.T)

In [514]:
def get_mean_cov(last_layer, cov_estimator="empirical"):
    mean = last_layer.mean(0)
    cov = torch.Tensor(get_cov_estimator(last_layer.detach().numpy(), cov_estimator))
    return mean, cov

In [515]:
mean, cov = get_mean_cov(last_layer, cov_estimator="LW")

0.510341562552002
[4533.74696871]


In [519]:
def mahalanobis(x, mean, cov):
    inv_cov = torch.inverse(cov)
    dist = (x - mean).T @ inv_cov @ (x - mean)
    return dist

In [520]:
# y = torch.rand(768)
manhalobis(y, mean, cov)

tensor(4533.7480, grad_fn=<DotBackward>)

In [ ]:
def get_score()

In [526]:
def get_roc(thresh)

SyntaxError: invalid syntax (<ipython-input-526-e08f3babd0a8>, line 1)

Reste à faire : comprendre ce qu'ils font avec les labels dans train_features, c'est là-dessus qu'ils calculent cov

In [142]:
t0 = time()
series = df_bae.original_text
Outputs = run_BERT(to_run[:8], output_hs=True)
print(time() - t0)

0.6506667137145996


In [143]:
a[0].hidden_states[12].shape

torch.Size([8, 256, 768])

Outputs

outputs = []
for idx in tqdm(range(2)):
    outputs.append(model(**tokenizer(to_run[idx*10:(idx+1)*10],max_length=256,return_tensors='pt',truncation=True,padding="max_length",\
          add_special_tokens=True,return_attention_mask=True)))


In [71]:
outputs.hidden_states[-1][:, 0, :].shape

torch.Size([2, 768])

a.attentions de shape 12\*batch_size\*12\*256\*256\
a.hidden_states de shape 13\*batch_size\*256\*768 

In [ ]:
mask = output.logits.argmax(1).tolist() != df_bae.original_output[:100]
otp = df_bae[:100][mask]
otp

NameError: name 'output' is not defined

-> Nos misclassifications sont plus grandes que le BERT qu'ils utilisent -> notre modèle est moins bon

### Méthode de classification des attaques
Pour l'instant pseudo-code dégueulasse pour structurer la pensée

In [155]:
class DetectionMethods:
    def __init__(self, method, attacks):
        ### Inputs:
        # method : distance used
        # attacks : a dataframe containing text, desired_output, and whether it's an attack or not
        self.method = method
        self.attacks = attacks

    def get_model_output(self):
        self.model_output = torch.softmax(
            model(
                **tokenizer(
                    self.attacks.text.tolist(), return_tensors="pt", padding=True
                )
            ).logits,
            dim=1,
        )

    def get_distance(self):
        if self.method == "Max_softmax":
            return np.max(self.model_output, axis=0)
        if self.method == "Kullback-Leibler":
            # TODO
            return
        if self.method == "Wasserstein":
            # Ca coûte rien de le rajouter hehe
            return wasserstein_distance(self.model_output, [0.25] * 4)
        if self.method == "My_Method":
            # TODO
            sorted_output = np.argsort(self.model_output, axis=0)
            return sorted_output[-1] - sorted_output[-2]
        if self.method == "Manhalobis":
            # TODO
            return

    def get_FPR(classifier):
        ### Computes FPR score
        return FPR_score

    def set_threshold(target=0.10, error=1e-5, n_iter=20):
        ### Finds by dichotomy a distance threshold such that the FPR equals a certain target (usually 10%)
        threshold = 0.5
        t_min = 0
        t_max = 1
        for iter in range(n_iter):
            FPR = self.FPR(classifier(threshold))
            if abs(FPR - target) > 1e-5:
                self.threshold = threshold
                return f"Successful convergence, threshold set to {self.threshold}"
            if FPR < 0.1:
                t_min = threshold
                threshold = (threshold + t_max) / 2
            else:
                t_max = threshold
                threshold = (threshold + t_min) / 2
        self.threshold = threshold
        return f"No convergence after {n_iter} iterations, threshold set to {self.threshold}, FPR = {FPR}"

In [144]:
def run_BERT(series, batch_size=10, output_hs=False, output_attentions=False):
    # We choose to run by batches to avoid kernel death
    tokens = tokenizer(
        series.tolist(),
        max_length=256,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        add_special_tokens=True,
        return_attention_mask=True,
    )
    num_batches = (len(series) - 1) // batch_size + 1
    Outputs = []
    for batch_idx in range(num_batches):  # Next line can probably be coded way cleaner
        window = [batch_idx * batch_size, (batch_idx + 1) * batch_size]
        Outputs.append(
            model(
                input_ids=tokens["input_ids"][window[0] : window[1]],
                attention_mask=tokens["attention_mask"][window[0] : window[1]],
                token_type_ids=tokens["token_type_ids"][window[0] : window[1]],
                output_hidden_states=output_hs,
                output_attentions=output_attentions,
            )
        )
    return Outputs
    # Outputs = torch.cat(outputs)
    # return Outputs

In [145]:
outputs = run_BERT(series[:11], output_hs=True, output_attentions=True)

In [146]:
# Idée : à terme tout mettre dans une classe, mettre des if last_layer : last_layer.append, etc...
# Et définir au début: if method = Manhalobis, set last_layer = True, etc...
def get_features(Outputs):
    # We wish to isolate a dataset containing in X the Manhalobis distance, and in Y the prediction
    last_layer = []
    pred = []
    softmax = []
    num_batches = len(Outputs)
    for batch_idx in range(num_batches):
        last_layer.append(Outputs[batch_idx].hidden_states[-1][:, 0, :])
        pred.append(Outputs[batch_idx].logits.argmax(1))
        softmax.append(torch.softmax(Outputs[batch_idx].logits, dim=1))
    last_layer = torch.cat(last_layer)
    pred = torch.cat(pred)
    softmax = torch.cat(softmax)
    return last_layer, pred, softmax

In [214]:
last_layer, pred, softmax = get_features(outputs)

In [513]:
def get_cov_estimator(x, name):
    if name == "LW":
        print(LedoitWolf().fit(x).shrinkage_)
        print(LedoitWolf().fit(x).mahalanobis(y.numpy().reshape(1, -1)))
        return LedoitWolf().fit(x).covariance_
    if name == "empirical":
        return np.cov(x.T)

In [78]:
get_mean_cov(x.T, estimator="MCD")[1].det()

tensor(1.5654e-22)

In [519]:
def mahalanobis(x, mean, cov):
    inv_cov = torch.inverse(cov)
    dist = (x - mean).T @ inv_cov @ (x - mean)
    return dist

In [520]:
# y = torch.rand(768)
manhalobis(y, mean, cov)

tensor(4533.7480, grad_fn=<DotBackward>)

In [ ]:
def get_score()

In [526]:
def get_roc(thresh)

SyntaxError: invalid syntax (<ipython-input-526-e08f3babd0a8>, line 1)

# FIRST TRY

In [ ]:
class project:
    load_model
    tokenize
    run_model
    get_data(type=train or type=test)
    get_features (existe déjà)
    preprocess (reduce_dim)
    get_mean_cov    
    get_attack_data
    get_score (implique mala)
    get_roc_curve(fpr_thresh=)
    
    

### Step 1: Loading the model and data

In [6]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-ag-news"
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "textattack/bert-base-uncased-ag-news"
)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
goal_function = UntargetedClassification(model_wrapper)
dataset = HuggingFaceDataset("ag_news", None, "test")

NameError: name 'transformers' is not defined

Separating train and test data

In [10]:
def get_data(name="ag_news"):
    dataset = load_dataset(name).shuffle(seed=0)
    # num_labels = dataset['train'].features['label'].num_classes
    return dataset["train"], dataset["test"]

In [11]:
train_data, test_data = get_data("ag_news")

NameError: name 'load_dataset' is not defined

Loading attacks?

### Step 2: Estimating covariance matrix on train set

In [13]:
def run_model(series, batch_size=10, output_hs=False, output_attentions=False):
    # We choose to run by batches to avoid kernel death
    tokens = tokenizer(
        series.tolist(),
        max_length=256,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        add_special_tokens=True,
        return_attention_mask=True,
    )
    num_batches = (len(series) - 1) // batch_size + 1
    Outputs = []
    for batch_idx in range(num_batches):  # Next line can probably be coded way cleaner
        window = [batch_idx * batch_size, (batch_idx + 1) * batch_size]
        Outputs.append(
            model(
                input_ids=tokens["input_ids"][window[0] : window[1]],
                attention_mask=tokens["attention_mask"][window[0] : window[1]],
                token_type_ids=tokens["token_type_ids"][window[0] : window[1]],
                output_hidden_states=output_hs,
                output_attentions=output_attentions,
            )
        )
    last_layer = []
    pred = []
    softmax = []
    num_batches = len(Outputs)
    for batch_idx in range(num_batches):
        if output_hs:
            last_layer.append(Outputs[batch_idx].hidden_states[-1][:, 0, :])
        pred.append(Outputs[batch_idx].logits.argmax(1))
        softmax.append(torch.softmax(Outputs[batch_idx].logits, dim=1))
    last_layer = torch.cat(last_layer)
    pred = torch.cat(pred)
    softmax = torch.cat(softmax)
    if output_hs:
        return last_layer, pred, softmax
    return pred, softmax
    # Outputs = torch.cat(outputs)
    # return Outputs

Charging last layer outputs

In [81]:
last_layer = run_model(train_data["text"], output_hs=True)[
    0
]  ### Looking into the last model layer

NameError: name 'train_data' is not defined

Missing : Preprocessing

Computing covariance matrix

In [88]:
estimators_dict = {"LW": LedoitWolf, "OAS": OAS, "MCD": MinCovDet}


def get_mean_cov(x, estimator="Empirical"):
    mean = x.mean(0)
    x = x.detach().numpy()
    if estimator == "Empirical":
        cov = np.cov(x)
    else:
        if estimator not in estimators_dict.keys():
            return "Wrong estimator, name must be in ['Empirical','LW','OAS','MCD',]"
        cov = torch.Tensor(estimators_dict[estimator]().fit(x).covariance_)
    return mean, cov

In [89]:
cov_estimator = "LW"
mean, cov = get_mean_cov(
    last_layer, estimator=cov_estimator
)  ### Computing the covariance matrix

### Step 3 : computing score on test set

Missing : Loading attacks (linking test_data with 1{is_attack})

In [ ]:
-> dataset 

Charging last layer outputs

In [81]:
last_layer, pred, _ = run_model(
    test_data["text"], output_hs=True
)  ### Looking into the last model layer

NameError: name 'train_data' is not defined

In [ ]:
def get_mahanalobis_score(train_data, test_data, last_layer, cov_estimator="LW"):
    train_sets_idx = get_shuffle_idx(train_data)
    Distance = []
    for idx in train_sets_idx:
        mean, cov = get_mean_cov(last_layer[train_sets_idx], estimator=cov_estimator)
        Distance.append(get_mahalanobis(test_data, mean, cov))
    Score = (
        np.array(Distance) * -0.5 + math.log(2 * math.pi) * mean.shape[0] * -0.5
    )  ### COMPRENDRE mean.shape[0]

Computing mahanalobis distance

In [ ]:
def get_shuffles(data,split_size=0.9,num_samples=100):


In [ ]:
train_sets = get_shuffles(test_data)

In [101]:
def get_mahalanobis(x, mean, cov):
    inv_cov = torch.inverse(cov)
    dist = (x - mean) @ inv_cov @ (x - mean).T
    return torch.diag(dist)

In [104]:
for data_shuffle in test_sets:
    dist = get_mahalanobis(last_layer, mean, cov)
    log_likelihood = dist * -0.5 + math.log(2 * math.pi) * mean.shape[0] * -0.5

tensor([157.2993, 156.8305, 159.5189, 158.5300, 157.0186, 157.2298, 157.0273,
        157.8824, 156.4181, 156.8805, 157.2887, 157.3431, 158.4945, 158.4800,
        157.6188, 157.0679, 158.6778, 157.9828, 158.7034, 158.1963])

(ici utiliser des train_set randomisés (90% du train_set à chaque fois), pour avoir différents mu,cov, ici fait pour une seul)

In [110]:
def get_score(x, mean, cov):
    score = (
        get_mahalanobis(x, mean, cov) * -0.5
        + math.log(2 * math.pi) * mean.shape[0] * -0.5
    )